# **Assignment 4**
**Francesco Emanuele Conti**

**N. Matricola: 0766488**


## **Scopo dell'assignment**
Lo scopo di questo esercizio è implementare un sistema di riconoscimento facciale "a mondo chiuso" basato su reti neurali convoluzionali, cercando di migliorare il lavoro svolto nell'Esercizio 2. Tale procedimento utilizzerà le librerie Tensorflow, Keras, OpenCV e NumPy per la costruzione di vari modelli di deep learning in grado di apprendere le identità di quattro persone tramite un approccio supervisionato, e conseguentemente provare a prevedere tali identità su un set di dati sconosciuto. In conclusione, si produrrà un'analisi dei modelli costruiti, cercando di mettere in evidenza le differenze su variabili quali:
- Categorical Accuracy.
- Number of Parameters.
- Validation Loss.

### **Procedimento seguito**
1. Importazione delle librerie necessarie e metodi utili.
2. Preparazione del dataset.
3. Suddivisione dei dati.
4. Progettazione e addestramento dei modelli.
5. Analisi dei risultati.
6. Verifica sul video di test.

#### Import delle librerie necessarie e metodi utili
Le librerie seguenti ci serviranno per la costruzione dei modelli, in particolare:
- **tensorflow** sta alla base della struttura di tutti i modelli di machine learning.
- **numpy** ci servirà per gestire le matrici di immagini.
- **cv2** la utilizziamo per l'apertura e il salvataggio delle immagini del dataset.
- **YOLO** è una rete neurale specializzata nell'object recognition. Utilizzeremo una sua variante (YoloV8-faces) per il face recognition.
- **keras** è la libreria specializzata nella progettazione bare-metal di reti neurali. La utilizzeremo per costruire una CNN "a mano".
- **matplotlib** la utilizzeremo come libreria per costruire grafici e visualizzare le immagini

In [2]:
import tensorflow as tf
import numpy as np
import cv2
from ultralytics import YOLO
from keras.applications import ResNet50, InceptionV3
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Resizing, Rescaling, BatchNormalizationV2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential, load_model
from Utils.live_predict import live_predict
from Utils.image_predict import image_predict
from keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
import os, random, shutil
import tqdm
import matplotlib.pyplot as plt

In [3]:
def show_accuracy_history(history):

    plt.plot(history.history["categorical_accuracy"], label="training")
    plt.plot(history.history["val_categorical_accuracy"], label="validation")
    plt.xticks(range(1, len(history.history['categorical_accuracy']) + 1))
    plt.xlabel("Epochs")
    plt.ylabel("Categorical Accuracy")
    plt.legend()
    plt.show()

def show_loss_history(history):
    training_loss = history.history['loss']
    validation_loss = history.history['val_loss']
    epochs = range(1, len(training_loss) + 1)

    # Tracciare il grafico
    plt.plot(epochs, training_loss, 'b-', label='training')
    plt.plot(epochs, validation_loss, 'orange', label='validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [4]:
def predict_image(model, dim):
        
    classi = ['Davide', 'Francesco', 'Gabriele', 'Stefano']

    while True:

        index_class = random.randint(0,3)
        chosen_class = classi[index_class]
        images_list = os.listdir(f"Dataset\\Original\\{chosen_class}\\")

        image_predict(f"Dataset\\Original\\{chosen_class}\\{images_list[random.randint(0,len(images_list)-1)]}", model, classi, dim=dim)




#### Preparazione del dataset
Le immagini di partenza sono contenute nella cartella ```Dataset/Original```. Queste immagini sono così suddivise:

- Dataset
    - Original
        - Davide (6000 immagini)
        - Francesco (6000 immagini)
        - Gabriele (6000 immagini)
        - Stefano (6000 immagini)

Le $6000$ immagini per candidato sono state ottenute tramite *frame capturing* su video di circa $200$ secondi $(3:20)$, registrati ad una risoluzione di $1280p \times 720p$ a $30$ FPS.

Su queste immagini è stato poi eseguito un algoritmo di **face recognition** tramite la rete neurale **YOLOv8** in modo da poter salvare solo le porzioni d'immagine contenenti le facce. 

Tale algoritmo, descritto nel seguito all'interno del metodo ```get_face```, si occupa di identificare la ROI (Region of Interest) dove si trova la faccia, riscalarla ad una dimensione di $64 \times 64$ pixels e salvarla nella sottocartella ```Dataset/Resized/nome_candidato```

In [5]:
def get_face(image_path):
    '''
    Questo metodo prende in input il percorso di un'immagine e restituisce la ROI contenete la faccia.
    
    Args:
        - image_path: il percorso relativo ad un'immagine.
    
    Output:
                
        - face: matrice numpy 64x64 uint8 contenente la faccia.
    '''

    yolo_face = YOLO("Utils/YoloFaceV8/yolov8n-face.pt")
    resize_and_rescale = Sequential([ 
                                        Resizing(64, 64, crop_to_aspect_ratio=True), 
                                    ])

    image = cv2.imread(image_path)

    #max_det=1 indica all'algoritmo di rilevare massimo 1 volto.
    #device=0 abilita l'utilizzo della computazione CUDA (togliere questo parametro se non e' disponibile)
    #conf=0.70 indica la soglia di confidence per l'identificazione di una faccia
    #verbose=False disabilita un output eccessivo sul terminale
    prediction = yolo_face.predict(source=image, max_det=1, device=0, conf=0.70, verbose=False)

    if len(prediction) == 1:
        ROIs = prediction[0].boxes  #la variabile boxes contiene i valori per identificare la ROI
        ROIs = ROIs.cpu()   # E' necessario copiare il tensore CUDA sulla CPU prima di poterlo convertire a numpy.
        ROIs = ROIs.numpy() # L'oggetto è in formato PyTorch, va prima convertito in NumPy per poterlo modificare.

        for roi in ROIs:

            x1 = int(roi.xyxy.tolist()[0][0])
            y1 = int(roi.xyxy.tolist()[0][1])
            x2 = int(roi.xyxy.tolist()[0][2])
            y2 = int(roi.xyxy.tolist()[0][3])

            face = image[y1:y2, x1:x2]          # prelevo la ROI dall'immagine originale
            face = resize_and_rescale(face)     # applico il layer sequenziale di keras per ridimensionare e riscalare l'immagine

            #l'oggetto face dopo resize_and_rescale e' un oggetto tf. Va riportato in numpy
            face_uint8 = (face.numpy()).astype(np.uint8)

            return face_uint8
    else: 
        return None

def build_dataset():
    '''
    Questo metodo costruisce una cartella Resized che contiene le facce di ogni candidato
    suddivise per classe.
    '''



    class_names = os.listdir("Dataset/Original") #Contiene i nomi delle classi (candidati)

    for cls in class_names:
        os.makedirs(f"Dataset/Resized/{cls}")   # Nella nuova cartella Resized, creo una directory per candidato

    for root, _, files in os.walk("Dataset/Original"):
        for file_name in tqdm(iterable=files, desc="Building RESIZED dataset..."):
            if file_name.endswith(".jpg"):
                file_path = os.path.join(root, file_name)  # Ottieni il percorso completo del file
                folder_name = os.path.dirname(file_path)   # Contiene il nome della cartella
            
                # eseguo l'algoritmo di face recognition per trovare una bounding box.
                face_found = get_face(file_path) 
                if face_found is not None: # se l'algoritmo ha trovato una faccia nell'immagine
                    cv2.imwrite(f"Dataset/Resized/{folder_name}/{file_name}", face_found)   #inserisco la faccia trovata nella cartella Resized relativa al candidato
    
    print("RESIZED dataset built.")

Al termine del codice appena descritto, ci ritroveremo con le seguenti directory:

- Dataset
    - Original

        - Davide (6000 immagini)
        - Francesco (6000 immagini)
        - Gabriele (6000 immagini)
        - Stefano (6000 immagini)  
  
    - Resized

        - Davide (~6000 ROI faces 64x64)
        - Francesco (~6000 ROI faces 64x64)
        - Gabriele (~6000 ROI faces 64x64)
        - Stefano (~6000 ROI faces 64x64)

In particolare, nell'assignment svolto, le ROI faces non sono ~6000 a candidato ma solo 5250. Questo è dovuto al fatto che sulle immagini originali dei vari candidati, YoloFaces non è riuscito ad identificare volti. A questo punto per un corretto equilibrio di addestramento al fine di evitare overfitting, si è deciso di ridurre il numero di immagini al minimo numero presente in tutte e quattro le classi.

### Suddivisione dei dati
Dopo aver costruito la directory ```Dataset/Resized``` si può procedere alla costruzione del vero e proprio dataset suddiviso in tre parti disgiunte:
- Training set contenente il 60% delle immagini totali.
- Validation set contenente il 50% delle immagini rimanenti.
- Test set contenente il 100% delle immagini rimanenti.

La suddivisione è quindi effettuata secondo la proporzione $(0.6, 0.2, 0.2)$.

E' importante sottolineare che il dataset da costruire deve contenere un numero bilanciato di immagini per classe, al fine di evitare overfitting o errori sulle metriche di valutazione.

In [6]:
def build_train_val_test():

    # Directory da cui recuperare le immagini da suddividere
    directory = "Resized\\"

    # Directory di destinazione per i set di addestramento, validazione e convalida
    train_dir = "Dataset\\Training\\"
    validation_dir = "Dataset\\Validation\\"
    test_dir = "Dataset\\Test\\"

    # Percentuale di immagini per i set
    train_percentage = 0.6
    validation_percentage = 0.2
    test_percentage = 0.2

    # Elenco delle classi
    classi = os.listdir(directory)

    # Itero su ogni classe, suddividendo le immagini per classe, in questo modo ne preservo l'equilibrio.
    for classe in classi:
        print(f"Elaboro la classe [{classe}]")

        classe_path = os.path.join(directory, classe)
        
        # Elenco delle immagini per la classe corrente
        immagini = os.listdir(classe_path)
        
        # Eseguo uno shuffle delle immagini in modo da non prenderle in maniera sequenziale (Frame vicini equivalgono ad immagini uguali)
        random.shuffle(immagini)
        
        # Calcolo le dimensioni dei set di addestramento, validazione e convalida
        num_immagini = len(immagini)
        num_train = int(train_percentage * num_immagini)
        num_validation = int(validation_percentage * num_immagini)
        num_test = num_immagini - num_train - num_validation
        
        # Divido le immagini nei tre set
        train_images = immagini[:num_train]
        validation_images = immagini[num_train:num_train+num_validation]
        test_images = immagini[num_train+num_validation:]
        
        # Creo le cartelle di destinazione se non esistono
        os.makedirs(os.path.join(train_dir, classe), exist_ok=True)
        os.makedirs(os.path.join(validation_dir, classe), exist_ok=True)
        os.makedirs(os.path.join(test_dir, classe), exist_ok=True)
        
        # Sposto le immagini nei rispettivi set
        for img in train_images:
            src_path = os.path.join(classe_path, img)
            dest_path = os.path.join(train_dir, classe, img)
            shutil.copy(src_path, dest_path)
        
        for img in validation_images:
            src_path = os.path.join(classe_path, img)
            dest_path = os.path.join(validation_dir, classe, img)
            shutil.copy(src_path, dest_path)
        
        for img in test_images:
            src_path = os.path.join(classe_path, img)
            dest_path = os.path.join(test_dir, classe, img)
            shutil.copy(src_path, dest_path)


Al termine del precedente codice avremo costruito tre directory: ```Training```, ```Validation``` e ```Test```. Ognuna contenente quattro sottodirectory (una per candidato) dove sono state suddivise secondo un rapporto $(0.6, 0.2, 0.2)$ le immagini provenienti dalla directory ```Resized```. A questo punto ci ritroveremo con la seguente struttura:

Dimensione per candidato del training set: $5250 * 0.6 = 3150$ immagini.

Dimensione per candidato del validation e test set: $5250 * 0.2 = 1050$ immagini.

- Dataset
    - Original

        - Davide (6000 immagini)
        - Francesco (6000 immagini)
        - Gabriele (6000 immagini)
        - Stefano (6000 immagini)  
  
    - Resized

        - Davide (~6000 ROI faces 64x64)
        - Francesco (~6000 ROI faces 64x64)
        - Gabriele (~6000 ROI faces 64x64)
        - Stefano (~6000 ROI faces 64x64)

    - Training

        - Davide (3150 ROI faces 64x64)
        - Francesco (3150 ROI faces 64x64)
        - Gabriele (3150 ROI faces 64x64)
        - Stefano (3150 ROI faces 64x64)
    
    - Validation

        - Davide (1050 ROI faces 64x64)
        - Francesco (1050 ROI faces 64x64)
        - Gabriele (1050 ROI faces 64x64)
        - Stefano (1050 ROI faces 64x64)
    
    - Test

        - Davide (1050 ROI faces 64x64)
        - Francesco (1050 ROI faces 64x64)
        - Gabriele (1050 ROI faces 64x64)
        - Stefano (1050 ROI faces 64x64)

### Progettazione e addestramento dei modelli
Come anticipato nell'introduzione, cercheremo di sviluppare dei modelli di rete neurale convoluzionale allo scopo di riconoscere le identità dei candidati. A tal fine si è scelto di utilizzare i seguenti parametri comuni per tutti i modelli:

1. La funzione di perdita è la ```categorical_crossentropy``` in quanto è particolarmente adatta per problemi in cui si desidera assegnare un'etichetta corretta ad un insieme discreto di classi, come nel caso del riconoscimento di identità.

2. La metrica di valutazione è la ```categorical_accuracy``` che misura la percentuale di campioni di dati per i quali il modello ha previsto correttamente l'etichetta di classe corretta rispetto all'etichetta di classe reale. Quando si lavora con un numero limitato di classi, come nel nostro caso, la categorical_accuracy fornisce una valutazione diretta delle capacità di classificazione del modello misurando l'efficacia globale del modello nel prevedere correttamente l'identità corrispondente in base ai dati di input.

3. Ottimizzatore del modello ```Adam``` che consente un adattamento dinamico del tasso di apprendimento durante il processo di addestramento ed essendo anche in grado di gestire efficacemente i gradienti rumorosi

Possiamo innanzitutto inizializzare il nostro dataset:

In [7]:
# il data generator viene utilizzato per generare al momento delle immagini aumentate.
datagen = ImageDataGenerator(
            rotation_range=10,              # Rotazione casuale che va dai -10° ai 10°
            shear_range=0.2,                # Deformazione bidimensionale massima del 20%
            brightness_range=[0.8, 1.5],    # Variazione della luminosità che va da 0.8 (più scura) a 1.5 (molto più chiara)
            horizontal_flip=True            # Flip orizzontale dell'immagine (immagine specchiata) possibile.
)

# costruisco due training set differenti:
# - uno con le immagini aumentate
# - uno senza le immagini aumentate

training_set_aug = datagen.flow_from_directory(
    directory="Dataset\\Training\\",
    class_mode="categorical",       # specificando la class_mode come categorical, non devo trasformare le label successivamente
    color_mode="rgb",               # sebbene cv2 operi in BGR, Tensorflow è ottimizzato per lavorare con canali RGB.
    target_size=(64, 64),
    batch_size=64,
    shuffle=True,
    seed=69,
    keep_aspect_ratio=True
)

training_set = image_dataset_from_directory(
    directory="Dataset\\Training\\",
    label_mode="categorical",       # stessa cosa di class_mode
    color_mode="rgb",
    image_size=(64, 64),
    batch_size=64,
    shuffle=True,
    seed=69,
    crop_to_aspect_ratio=True
)

# validation e training set non necessitano di data augmentation

validation_set = image_dataset_from_directory(
    directory="Dataset\\Validation\\",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(64, 64),
    batch_size=64,
    shuffle=True,
    seed=69,
    crop_to_aspect_ratio=True
)

test_set = image_dataset_from_directory(
    directory="Dataset\\Test\\",
    label_mode="categorical",
    color_mode="rgb",
    image_size=(64, 64),
    batch_size=64,
    crop_to_aspect_ratio=True
)


Found 12600 images belonging to 4 classes.
Found 12600 files belonging to 4 classes.
Found 4200 files belonging to 4 classes.
Found 4200 files belonging to 4 classes.


#### ResNet50 pre-addestrata
Il primo modello che vogliamo testare consiste in una **ResNet** backbone pre addestrata sul dataset **ImageNet** a cui vengono inseriti due strati densi rispettivamente da 64 e 4 unità. Quest'ultimo layer da 4 è anche l'uscita del nostro modello.

In [8]:
def resNet50_pretrained():
    '''
    Input: 64x64x3 BGR centered image
    Parameters: 24.112.324 (Trainable: 1.579.332)
    Disabled layers: All except the last 5
    Batch size: 256
    Max epochs: 100
    Output: Double dense layers (64 to 4)
    '''

    #ResNet richiede uno specifico preprocessing sull'input (Fonte: https://keras.io/api/applications/resnet/#resnet50-function)
    from keras.applications.resnet import preprocess_input

    print("Preprocessing Training Set...")
    for i, element in enumerate(training_set_aug):
        preprocess_input(element[0])
        if i == len(training_set_aug) - 1:
            break
    print("Preprocessing Validation Set...")
    for element in validation_set:
        preprocess_input(element[0])
    print("Preprocessing Test Set...")
    for element in test_set:
        preprocess_input(element[0])


    backbone = ResNet50(include_top=False, weights='imagenet', input_shape=(64,64,3))
    
    # disabilito l'addestramento di tutti i layer della backbone, tranne gli ultimi cinque
    for layer in backbone.layers[:-5]:
        layer.trainable = False

    callback =  [   
                    EarlyStopping(patience=7, restore_best_weights=True), 
                    ModelCheckpoint(filepath='Models\\ResNet\\pre_trained\\ResNet50_pretrained.{epoch:02d}-{val_loss:.2f}.h5') 
                ]

    model = Sequential()
    model.add(backbone)
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    model.summary()

    input("Premi un tasto per iniziare l'addestramento...")

    history = model.fit(training_set_aug, batch_size=256, epochs=100, callbacks=callback, validation_data=validation_set)

    # stampiamo a video l'accuratezza e la loss
    loss, accuracy = model.evaluate(test_set)
    print("Test Loss: ", loss)
    print("Test Accuracy: ", accuracy)

    show_accuracy_history(history)
    show_loss_history(history)

#resNet50_pretrained()


#model = load_model("Models\ResNet\pre_trained\ResNet50_pretrained.10-0.02.h5")
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 64)
#predict_image(model)

#loss, accuracy = model.evaluate(test_set)
#print("Test Loss: ", loss)
#print("Test Accuracy: ", accuracy)


Possiamo osservare i risultati di questo modello nei seguenti due grafici, di cui il primo rappresenta l'accuratezza, mentre il secondo l'andamento della funzione di perdita.

![Accuracy graph](Utils/Graphs/ResNet50_pretrained/accuracy.png)        ![Loss graph](Utils/Graphs/ResNet50_pretrained/loss.png)

Si può osservare che il modello mostra un comportamento di generalizzazione fino alla fase di addestramento all'ottava epoca. Tuttavia, successivamente, si nota che nonostante l'accuracy durante l'addestramento continui ad aumentare, l'accuracy durante la fase di validazione tende a oscillare, non mantenendo più un andamento lineare. Questo comportamento suggerisce la presenza di overfitting, in cui il modello potrebbe aver imparato a memorizzare i dati di addestramento piuttosto che generalizzare e adattarsi ai dati nuovi o di validazione.

Il grafico dell'accuratezza suggerisce che le epoche migliori possano essere la settima, l'ottava e la decima. D'altra parte però il grafico della funzione di perdita ci mostra un andamento oscillatorio già a partire dalla nona epoca, questo ci fa escludere principalmente tutte le epoche a partire dalla nona.

Come verifica finale abbiamo osservato il comportamento del modello sul test set e sul video di prova, ottenendo i seguenti risultati:

- Settima epoca:
    - loss: 0.0257
    - categorical_accuracy: 0.9936
  
- Ottava epoca: 
    - loss: 0.0140 
    - categorical_accuracy: 0.9957

- Decima epoca:
    - loss: 0.0153 
    - categorical_accuracy: 0.9964

Infine abbiamo verificato tutte e tre le epoche sul video finale di test, dove si è riscontrato che la settima epoca è la migliore nel riconoscimento delle identità.

#### ResNet50 senza pre addestramento
Il secondo modello testato è la stessa ResNet di prima, rimuovendo il pre addestramento. Il numero di parametri addestrabili diventa $24,059,204$.

Purtroppo, in questo caso la rete diventa troppo complessa da gestire e non si è riuscito ad ottenere dei risultati soddisfacenti. Le modifiche sull'architettura sono state apportate nei seguenti parametri:

- Preprocessing del training set.
- Aggiunta, modifica o rimozione di Layer Dense sugli strati finali.
- Modifica del learning rate dell'ottimizzatore.
- Modifica del parametro patience dell'Early Stopping.
- Aggiunta di Batch Normalization.
- Aggiunta di layer di Dropout e relativa configurazione.
- Modifica del batch size.

Dopo vari tentativi la rete continuava ad andare in overfitting, oppure non riusciva ad apprendere correttamente le identità. Si riporta un esempio di history:

![History ResNet50_Naive](Utils/Graphs/ResNet50_naive/accuracy.png)     ![History ResNet50_Naive](Utils/Graphs/ResNet50_naive/loss.png)

Dal grafico soprastante si possono notare numerosi salti di accuratezza, anche nell'ordine del 70%, sintomo che la rete non riusciva ad apprendere correttamente.

Si riporta comunque il codice utilizzato per l'addestramento della rete che ha prodotto tale grafico:

In [9]:
def resNet50_no_pretrained():
    '''
    Input: 64x64x3 BGR centered image
    Parameters: 24.112.324 (Trainable: 24,059,204)
    Batch size: 256
    Max epochs: 100
    Output: Double dense layers (64 to 4)
    '''

    #ResNet richiede uno specifico preprocessing sull'input (Fonte: https://keras.io/api/applications/resnet/#resnet50-function)
    from keras.applications.resnet import preprocess_input


    print("Preprocessing Training Set...")
    for i, element in enumerate(training_set_aug):
        preprocess_input(element[0])
        if i == len(training_set_aug) - 1:
            break
    print("Preprocessing Validation Set...")
    for element in validation_set:
        preprocess_input(element[0])
    print("Preprocessing Test Set...")
    for element in test_set:
        preprocess_input(element[0])


    backbone = ResNet50(include_top=False, weights=None, input_shape=(64,64,3))
    
    callback =  [
                    EarlyStopping(patience=5, restore_best_weights=True),
                    ModelCheckpoint(filepath='Models\\ResNet\\naive\\ResNet50_naive.{epoch:02d}-{val_loss:.2f}.h5') 
                ]

    model = Sequential()
    model.add(backbone)
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    model.summary()

    input("Premi un tasto per iniziare l'addestramento...")

    history = model.fit(training_set, batch_size=256, epochs=100, callbacks=callback, validation_data=validation_set)

    # stampiamo a video l'accuratezza e la loss
    loss, accuracy = model.evaluate(test_set)
    print("Test Loss: ", loss)
    print("Test Accuracy: ", accuracy)

    show_accuracy_history(history)
    show_loss_history(history)


#resNet50_no_pretrained()


#### InceptionV3
Il prossimo modello che vogliamo addestrare consiste in una backbone InceptionV3 pre addestrata sul dataset ImageNet. In particolare, vanno prima specificati alcuni punti:

- Il modello Inception accetta come minime dimensioni in input $(75,75,3)$.
- Il modello è pre addestrato tramite ImageNet.
- I layer vengono tutti freezati tranne gli ultimi 5.
- Il modello accetta come dataset delle immagini riscalate nell'intervallo $[-1,1]$, per questo motivo è stato inserito il suo metodo di preprocessing.

Come verrà specificato in seguito, il modello costruito presenta due varianti:
- La variante iniziale consiste in due strati densi da 64 e 4 unità. Il numero di parametri totali è $21.934.180$ con $131.396$ parametri addestrabili
- La variante ottimizzata consiste in tre strati densi da 128, 64 e 4 unità, con l'aggiunta di un layer di dropout() tra il layer da 64 e quello da 4. In questo caso il numero totale di parametri è di $22.073.572$ con un numero di parametri addestrabili pari a $270.788$, dovuti all'aggiunta del nuovo layer dense.

In [10]:
def inceptionV3_pretrained():

    #InceptionV3 richiede uno specifico preprocessing sull'input (Fonte: https://keras.io/api/applications/inceptionv3/)
    from keras.applications.inception_v3 import preprocess_input
    
    training_set_aug = datagen.flow_from_directory(
    directory="Dataset\\Training\\",
    class_mode="categorical",       # specificando la class_mode come categorical, non devo trasformare le label successivamente
    color_mode="rgb",               # sebbene cv2 operi in BGR, Tensorflow è ottimizzato per lavorare con canali RGB.
    target_size=(75, 75),           # InceptionV3 richiede delle immagini 75x75
    batch_size=64,
    shuffle=True,
    seed=69,
    keep_aspect_ratio=True
    )

    training_set = image_dataset_from_directory(
        directory="Dataset\\Training\\",
        label_mode="categorical",       # stessa cosa di class_mode
        color_mode="rgb",
        image_size=(75, 75),
        batch_size=64,
        shuffle=True,
        seed=69,
        crop_to_aspect_ratio=True
    )

    # validation e training set non necessitano di data augmentation

    validation_set = image_dataset_from_directory(
        directory="Dataset\\Validation\\",
        label_mode="categorical",
        color_mode="rgb",
        image_size=(75, 75),
        batch_size=64,
        shuffle=True,
        seed=69,
        crop_to_aspect_ratio=True
    )

    test_set = image_dataset_from_directory(
        directory="Dataset\\Test\\",
        label_mode="categorical",
        color_mode="rgb",
        image_size=(75, 75),
        batch_size=64,
        crop_to_aspect_ratio=True
    )


    print("Preprocessing Training Set...")
    for i, element in enumerate(training_set_aug):
        preprocess_input(element[0])
        if i == len(training_set_aug) - 1:
            break
    print("Preprocessing Validation Set...")
    for element in validation_set:
        preprocess_input(element[0])
    print("Preprocessing Test Set...")
    for element in test_set:
        preprocess_input(element[0])


    # Costruzione del modello

    backbone = InceptionV3(include_top=False, weights='imagenet', input_shape=(75,75,3))

    callback =  [
                    EarlyStopping(patience=5, restore_best_weights=True),
                    ModelCheckpoint(filepath='Models\\Inception\\pre_trained_96\\InceptionV3_pretrained_96.{epoch:02d}-{val_loss:.2f}.h5') 
                ]
    
    # disabilito l'addestramento di tutti i layer della backbone, tranne gli ultimi cinque
    for layer in backbone.layers[:-5]:
        layer.trainable = False

    model = Sequential()
    model.add(backbone)
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))    # rimuovere questo strato per ottenere il modello originale di Inception
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.25))                    # rimuovere questo strato per ottenere il modello originale di Inception
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    model.summary()

    input("Premi un tasto per iniziare l'addestramento...")

    history = model.fit(training_set, batch_size=256, epochs=100, callbacks=callback, validation_data=validation_set)

    # stampiamo a video l'accuratezza e la loss
    loss, accuracy = model.evaluate(test_set)
    print("Test Loss: ", loss)
    print("Test Accuracy: ", accuracy)

    show_accuracy_history(history)
    show_loss_history(history)


#inceptionV3_pretrained()

#model = load_model("Models\\Inception\\pre_trained_128\\InceptionV3_pretrained_96.15-0.12.h5")
#predict_image(model, dim=75)
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 75)

Il modello appena descritto ha prodotto i seguenti risultati:

![Inception Accuracy](Utils/Graphs/Inception/accuracy.png)          ![Inception Loss](Utils/Graphs/Inception/loss.png)

Le migliori epoche sembrano essere la quinta e la decima, in quanto provengono da un intorno in cui l'accuratezza sui dati di validazione è in costante miglioramento, seguendo in maniera lineare l'accuratezza sul dataset di addestramento.

Si può osservare in generale che il modello tende ad un costante miglioramento tranne nell'intorno dell'ottava epoca. Questo può essere dovuto a varie cause, le più probabili possono essere:

- Overfitting: Il modello si è adattato troppo ai dati precedenti all'ottava epoca e quando ha dovuto predire dei dati nuovi l'accuracy è calata.
- Discrepanza tra i dati: La data augmentation utilizzata ha impostato come parametri shear (distorsione) e brightness (0.6 a 1.5), questo può aver causato un addestramento su immagini troppo 'spinte', abbassando la precisione del modello.

#### InceptionV3 seconda variante

Per evitare ciò, si è deciso di modificare la struttura del modello, aggiungendo un secondo strato denso da 128 unità e un layer di dropout(0.25) per cercare di ridurre l'overfitting. 

I risultati sono stati i seguenti:

![Inception 128 Accuracy](Utils/Graphs/Inception128/accuracy.png)       ![Inception 128 Loss](Utils/Graphs/Inception128/loss.png)

In questo caso, come è possibile notare, l'accuratezza e la funzione di perdita sono migliori sul set di validazione che sul set d'addestramento fino alla dodicesima epoca. Questo potrebbe essere sintomo di underfitting, ed infatti provando a predire le classi su dati di test, si è notato che l'accuratezza non era superiore ad un 65%.

Dalla dodicesima epoca in poi il modello è andato in costante miglioramento, trovando, secondo le valutazioni sui dati di test e sul video finale, la migliore accuratezza alla quindicesima epoca, dove i valori assunti dalla funzione di perdita sul dataset d'addestramento che su quello di validazione si equivalgono.

#### CustomNet
Infine si è provato a costruire una semplice rete neurale convoluzionale "bare-metal" che cercasse di riprodurre i risultati delle reti appena presentate.

Come prima prova si è costruito un modello sequenziale a 4 strati di profondità, dove ogni strato raddoppia il numero di mappe di features catturate.

##### Architettura proposta

1. La rete inizia con un layer Conv2D con 32 filtri di dimensione (3, 3). Questo layer applica la convoluzione su un'immagine in input per catturare le caratteristiche di basso livello, come bordi, linee e texture. 

2. Successivamente, viene utilizzato un layer di MaxPooling2D con dimensioni di pooling (2, 2), che riduce le dimensioni spaziali dell'immagine di input e riduce il numero di parametri da apprendere aiutando ad estrarre le caratteristiche salienti dall'immagine riducendo la dimensionalità. 

3. Viene poi aggiunto un altro layer Conv2D con 64 filtri di dimensione (3, 3), seguito da un ulteriore layer di MaxPooling2D. Questi layer consentono di catturare caratteristiche più complesse rispetto ai precedenti layer, consentendo alla rete di apprendere rappresentazioni di livello superiore. Il processo viene ripetuto con un ulteriore layer Conv2D con 128 filtri di dimensione (3, 3) ed un ultimo layer Conv2D con 256 filtri di dimensione (3, 3)

4. Dopo aver eseguito le operazioni di convoluzione e di max pooling, viene utilizzato un layer Flatten per convertire l'output in un vettore unidimensionale, in modo che possa essere passato a un classificatore finale  che consiste in un layer Dense con 128 unità nascoste e attivazione "relu". Questo contribuisce a combinare le caratteristiche apprese precedentemente in una rappresentazione compatta e significativa. Infine, viene aggiunto un layer Dense con 4 unità di output e attivazione "softmax", che produce le probabilità di appartenenza alle diverse classi di identità.

Come callback e parametri di compilazione, si è scelto di utilizzare gli stessi dei precedenti modelli in modo da non alterare il confronto finale. L'unica differenza consiste nel numero di epoche da attendere prima di avviare l'EarlyStopping, passando da 5 a 3.

In [11]:
def customNetV1():
    '''
    Input: 64x64x3 RGB
    Parameters: 520,132
    Disabled layers: None
    Batch size: 256
    Max epochs: 100
    Output: Double dense layers (128 to 4)
    ''' 

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    callback =  [
                    EarlyStopping(patience=3, restore_best_weights=True),
                    ModelCheckpoint(filepath='Models\\CustomNet\\V1\\customNetV1.{epoch:02d}-{val_loss:.2f}.h5') 
                ]
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    model.summary()

    input("Premi un tasto per iniziare l'addestramento...")

    history = model.fit(training_set_aug, batch_size=256, epochs=100, callbacks=callback, validation_data=validation_set)

    # stampiamo a video l'accuratezza e la loss
    loss, accuracy = model.evaluate(test_set)
    print("Test Loss: ", loss)
    print("Test Accuracy: ", accuracy)

    show_accuracy_history(history)
    show_loss_history(history)


#customNetV1()
#model = load_model("Models\\CustomNet\\V1\\customNetV1.10-0.01.h5")
#predict_image(model, dim=64)
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 64)

I risultati di questo modello possono essere riassunti nello storico della categorical accuracy e dell'errore:

![Accuracy CustomNetV1](Utils/Graphs/CustomNet/V1/accuracy.png)         ![Loss CustomNetV1](Utils/Graphs/CustomNet/V1/loss.png)

Sebbene l'andamento sembri lineare e costante, si è notato sul video di test un pesante overfitting verso le classi ```Stefano``` e ```Francesco```, segno che la semplicità di questa rete non riesce a catturare le feature necessarie a distinguere le classi. A questo proposito è stata costruita una seconda variante che cercasse di rimediare a questo problema:

#### CustomNet V2
La seconda variante cerca di ridurre il problema dell'overfitting, cercando di catturare maggiormente i dettagli sui contorni delle facce e sulla loro forma. In particolare sono state apportate le seguenti modifiche:

##### Architettura proposta

1. La rete inizia con un layer Conv2D con 32 filtri di dimensione (7, 7). L'uso di filtri di dimensioni maggiori consente alla rete di catturare caratteristiche più ampie e complesse nell'immagine di input. Inoltre, è stato aggiunto un kernel_regularizer con una penalità di regolarizzazione L2 (0.01) per ridurre il rischio di overfitting. Questo filtro cerca di catturare maggiormente le caratteristiche sui bordi delle facce.

2. Successivamente, viene utilizzato un layer di MaxPooling2D con dimensioni di pooling (2, 2) per ridurre la dimensionalità dell'immagine e selezionare le caratteristiche più salienti.

3. Viene poi aggiunto un altro layer Conv2D con 64 filtri sempre di dimensione (7, 7). In questo caso si è voluto provare ad acquisire maggiori informazioni circa la forma della faccia, cercando di distinguere meglio le caratteristiche comuni ai quattro candidati.

4. Per contrastare ulteriormente l'overfitting, è stato inserito un layer di Dropout con un tasso del 25% in modo da impedire alla rete di fare affidamento eccessivo sulle caratteristiche appena apprese sui bordi e forme particolari, migliorando la generalizzazione.

5. Dopodiché si è aumentato di quattro volte il numero di filtri presenti nella successiva convoluzione, passando da 64 a 256 filtri di dimensione (5,5) e poi dopo un layer di MaxPooling, un ulteriore layer da 512 filtri di dimensione (3,3). Questo allo scopo di addestrare il modello non solo sulla forma della faccia, ma aumentando l'affidamento sulle caratteristiche particolari del volto come il colore della pelle, piccole imperfezioni, forme degli occhi, naso e bocca, etc.

6. Dopo le operazioni di convoluzione e pooling, l'output viene sempre appiattito tramite un layer Flatten per preparare i dati e passarli al classificatore.

7. Il classificatore consiste nel solito layer Dense a 128 unità con attivazione RELU, seguite da un layer di Dropout con un tasso del 25% per aiutare a ridurre la dipendenza da caratteristiche specifiche.

8. Infine il layer Dense finale con attivazione softmax per la classificazione delle identità.

In [ ]:
def customNetV2():
    '''
    Input: 64x64x3 RGB
    Parameters: 3,334,212 
    Disabled layers: None
    Batch size: 256
    Max epochs: 100
    Output: Double dense layers (128 to 4)
    ''' 
    
    model = Sequential()
    model.add(Conv2D(32, (7, 7), activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (7, 7), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (5, 5), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, (3, 3), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation='softmax'))

    callback =  [
                    EarlyStopping(patience=3, restore_best_weights=True),
                    ModelCheckpoint(filepath='Models\\CustomNet\\V2\\customNetV2.{epoch:02d}-{val_loss:.2f}.h5') 
                ]
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    model.summary()

    input("Premi un tasto per iniziare l'addestramento...")

    history = model.fit(training_set_aug, batch_size=256, epochs=100, callbacks=callback, validation_data=validation_set)

    # stampiamo a video l'accuratezza e la loss
    loss, accuracy = model.evaluate(test_set)
    print("Test Loss: ", loss)
    print("Test Accuracy: ", accuracy)

    show_accuracy_history(history)
    show_loss_history(history)


#customNetV2()
#model = load_model("Models\\CustomNet\\V2\\customNetV2.21-0.37.h5")
#predict_image(model, dim=64)
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 64)

I risultati sono mostrati nel seguente storico:


![Accuracy CustomNetV2](Utils/Graphs/CustomNet/V2/accuracy.png)             ![Accuracy CustomNetV2](Utils/Graphs/CustomNet/V2/loss.png)

Nell'ultima versione del modello, si è notato un miglioramento costante delle prestazioni dopo le prime nove epoche, con un andamento lineare che indica una migliore capacità di predizione sui dati di validazione. Tuttavia, a partire dall'ventiduesima epoca, si è osservato un fenomeno crescente di overfitting, in cui il modello tende a memorizzare e adattarsi eccessivamente ai dati di addestramento, perdendo la capacità di generalizzare correttamente su nuovi dati.

Nonostante ciò, è importante sottolineare che rispetto alla versione precedente del modello, la ventunesima epoca ha mostrato un notevole miglioramento nella capacità di predizione. Ciò suggerisce che le modifiche apportate alla rete abbiano contribuito a migliorare le sue prestazioni iniziali.

### Analisi dei risultati

In conclusione, possiamo confrontare i vari modelli progettati e verificare quale sia il miglior approccio al compito di identity recognition che ci è stato posto. 

Di seguito un breve riassunto delle reti costruite:

1. **ResNet50 pre addestrata**: Rete neurale con backbone ResNet50 pre addestrata sul dataset ImageNet dove sono stati congelati dall'addestramento tutti i layer tranne gli ultimi 5.

2. **ResNet50 naive**: Rete neurale con backbone ResNet50 senza pre addestramento con layer di dropout.

3. **InceptionV3 pre addestrata**: Rete neurale con backbone InceptionV3 pre addestrata sul dataset ImageNet dove sono stati congelati tutti i layer tranne gli ultimi 5.
    1. Prima variante: La prima variante consiste in due strati finali densi da 64 e 4 unità.
    2. Seconda variante: La seconda variante consiste in tre strati finali densi da 128, 64 e 4 unità con l'aggiunta di un layer di Dropout per contrastare il fenomeno di overfitting.
    


4. **CustomNet**: Rete neurale convoluzionale fatta a mano.
    1. Prima variante: La prima variante è una semplice rete di profondità quattro che presenta overfitting accentuato.
    2. Seconda variante: La seconda variante è una versione altamente modificata negli strati convoluzionali e con l'aggiunta di regolarizzatori L2 e layer di dropout.


Mostriamo un confronto tra le varie architetture:

| Modello          | N. parametri | N. parametri addestrabili | Test Loss | Test Accuracy | Profondità | Input Size | Preprocessing necessario | Epoca migliore |
|------------------|--------------|---------------------------|-----------|---------------|------------|------------|--------------------------|----------------|
| ResNet50 PA*     | 24.112.324   | 1.579.332                 |  0.0257   |    0.9936     | 50 layers* | 64x64 BGR  | SI                       | 7              |
| ResNet50 Naive   | 24.112.324   | 24,059,204                |   N.Q*    |    N.Q*       | 50 layers* | 64x64 BGR  | SI                       | N.Q*           |
| InceptionV3 1 PA*| 21.934.180   | 131.396                   |  0.2300   |    0.9511     | 48 layers* | 75x75 RGB  | SI                       | 10             |
| InceptionV3 2 PA*| 21.934.180   | 270.788                   |  0.1320   |    0.9552     | 48 layers* | 75x75 RGB  | SI                       | 15             |
| CustomNet 1      | 520,132      | 520,132                   |  0.0124   |    0.9971     | 10 layers* | 64x64 RGB  | NO                       | N.Q*           |
| CustomNet 2      | 3,334,212    | 3,334,212                 |  0.3777   |    0.9773     | 10 layers* | 64x64 RGB  | NO                       | 21             |

*PA*: Pre addestrata.
*N.Q.*: Non qualificato.


Come possiamo osservare dalla misurazione della ```categorical accuracy``` effettuata sul test set, tutte le reti mostrano risultati eccellenti. Tuttavia, è importante fare attenzione a non cadere nell'errore di considerare questi risultati come indicativi di prestazioni ottimali, in quanto questi sono fortemente influenzati dall'overfitting. Tale problema è causato principalmente da due motivi:

1. Struttura del dataset utilizzato: il dataset è stato suddiviso in tre parti: il training set, il validation set e il test set, ma tutti i frame sono stati prelevati dalla stessa cartella. Ciò significa che con una probabilità molto alta, il test set contiene frame praticamente identici a quelli presenti nel validation set e nel training set. 

2. Ridotta quantità di dati utilizzati durante l'addestramento: Sono state utilizzate sono 3150 immagini per candidato, molte delle quali praticamente identiche, essendo prelevate da frame in successione.

Nonostante ciò, il miglior modello, secondo i risultati sperimentati è  l'InceptionV3 2, poiché ha mostrato i risultati più promettenti sia nel video di test che in prove supplementari condotte sul dataset originale (non croppato).


In conclusione, l'utilizzo delle reti neurali convoluzionali per il riconoscimento di identità ha dimostrato risultati promettenti. Queste sono state in grado di apprendere estraendo automaticamente caratteristiche rilevanti dalle immagini, consentendo di identificare con una discreta precisione le identità presenti nei dati di input. E' però importante considerare alcune sfide associate all'addestramento e all'utilizzo di queste reti. L'overfitting è un rischio da tenere in considerazione, soprattutto quando si dispone di dataset limitati o altamente simili tra i set di addestramento, validazione e test.

### Prova sul video finale

Per verificare visivamente quando scritto su questo notebook, è possibile eseguire il seguente codice:

In [ ]:
# ResNet 50 pre addestrata
#model = load_model("Models\\ResNet\\pre_trained\\ResNet50_pretrained.07-0.02.h5")
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 64)

# InceptionV3 classic
#model = load_model("Models\\Inception\\pre_trained\\InceptionV3_pretrained.10-0.14.h5")
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 75)

# InceptionV3 128
#model = load_model("Models\\Inception\\pre_trained_128\\InceptionV3_pretrained_128.15-0.12.h5")
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 75)

# CustomNet 1
#model = load_model("Models\\CustomNet\\V1\\customNetV1.07-0.01.h5")
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 64)

# CustomNet 2
#model = load_model("Models\\CustomNet\\V2\\customNetV2.21-0.37.h5")
#live_predict(model, ['Davide', 'Francesco', 'Gabriele', 'Stefano'], 1, 64)